In [0]:
dbutils.widgets.text('catalog_name', "")

In [0]:
catalog_name = dbutils.widgets.get("catalog_name")

In [0]:
df = spark.read.format('json')\
    .option('header', True)\
    .load(f'/Volumes/{catalog_name}/big_mart/mart_csv/minifigs.json')

df.display()

In [0]:
df.count()

In [0]:
from pyspark.sql.functions import explode
# faltaning the results array column using explode 
df2 = (
    df.withColumn("explodedArray", explode(df.results))
)

display(df2)

In [0]:
# this code will also use to use sql on this df2 and in that func we have to pass name of the df
df2.createOrReplaceTempView("myData")

In [0]:
%sql

SELECT 
  explodedArray.last_modified_dt AS LastModifiedOrignal,
  to_date(explodedArray.last_modified_dt) as LastModifiedDate,
  to_timestamp(explodedArray.last_modified_dt) as LastModifiedTimestamp,
  explodedArray.name AS Name,
  CASE 
    WHEN upper(explodedArray.name) LIKE '%TOY%' THEN 'Toy'
    WHEN upper(explodedArray.name) LIKE '%DROID%' THEN 'Droid'
    ELSE 'Other'
  END AS MinifigType,
  cast(explodedArray.num_parts as int) AS NumberOfParts,
  explodedArray.set_img_url AS ImageURL,
  explodedArray.set_num AS SetNumber,
  explodedArray.set_url AS SetURL
FROM 
 myData

In [0]:
_sqldf\
    .write\
    .format('delta')\
    .save(f'/Volumes/{catalog_name}/gold/goldvolume/big_mart')

In [0]:
delta_df = spark.read.format('delta').load(f'/Volumes/{catalog_name}/gold/goldvolume/big_mart/')

display(delta_df)

In [0]:
display(dbutils.fs.ls(f"/Volumes/{catalog_name}/gold/goldvolume/big_mart/"))


In [0]:
%sql
CREATE DATABASE PiotrT_Demo

Internal table is created in the hive/meatastore to store the data in delta format in databricks volumn which is mangaed by databricks it self. 

It is not recomentded to store data in databricks

In [0]:
%sql
CREATE TABLE PiotrT_Demo.Minifigs
AS
SELECT 
  explodedArray.last_modified_dt AS LastModifiedOrignal,
  to_date(explodedArray.last_modified_dt) as LastModifiedDate,
  to_timestamp(explodedArray.last_modified_dt) as LastModifiedTimestamp,
  explodedArray.name AS Name,
  CASE 
    WHEN upper(explodedArray.name) LIKE '%TOY%' THEN 'Toy'
    WHEN upper(explodedArray.name) LIKE '%DROID%' THEN 'Droid'
    ELSE 'Other'
  END AS MinifigType,
  cast(explodedArray.num_parts as int) AS NumberOfParts,
  explodedArray.set_img_url AS ImageURL,
  explodedArray.set_num AS SetNumber,
  explodedArray.set_url AS SetURL
FROM 
 myData

In [0]:
%sql
SELECT * FROM PiotrT_Demo.Minifigs

## External Table to store data on our storage account


In [0]:
%sql
CREATE TABLE PiotrT_Demo.Minifigs_External
--  LOCATION 'external_location_path'
AS
SELECT 
  explodedArray.last_modified_dt AS LastModifiedOrignal,
  to_date(explodedArray.last_modified_dt) as LastModifiedDate,
  to_timestamp(explodedArray.last_modified_dt) as LastModifiedTimestamp,
  explodedArray.name AS Name,
  CASE 
    WHEN upper(explodedArray.name) LIKE '%TOY%' THEN 'Toy'
    WHEN upper(explodedArray.name) LIKE '%DROID%' THEN 'Droid'
    ELSE 'Other'
  END AS MinifigType,
  cast(explodedArray.num_parts as int) AS NumberOfParts,
  explodedArray.set_img_url AS ImageURL,
  explodedArray.set_num AS SetNumber,
  explodedArray.set_url AS SetURL
FROM 
 myData

In [0]:
%sql
select * from PiotrT_Demo.Minifigs_External

In [0]:
%sql
SELECT count(*) FROM piotrt_demo.minifigs_external;

In [0]:
%sql
INSERT INTO piotrt_demo.minifigs_external(
    LastModifiedOrignal,
    LastModifiedDate,
    LastModifiedTimestamp,
    Name,
    MinifigType,
    NumberOfParts,
    ImageURL,
    SetNumber,
    SetURL
)
VALUES(
  "2020-05-27T21:47:00.694941Z",
  "2020-05-16",
  "2020-05-16T05:47:53.181+00:00",
  "New minifig 1",
  "Other",
  4,
  null,
  "fig-xxxx1",
  null
)

In [0]:
%sql
SELECT * FROM piotrt_demo.minifigs_external 
WHERE SetNumber = 'fig-xxxx1'


In [0]:
%sql
UPDATE 
  piotrt_demo.minifigs_external
SET 
  Name = 'Updated Minifigs'
WHERE SetNumber = 'fig-xxxx1'
    

In [0]:
%sql
DELETE FROM
  piotrt_demo.minifigs_external
WHERE 
  SetNumber = 'fig-xxxx1'

### Id bigint GENERATED ALWAYS AS IDENTITY,    -- automatically generates id for each new record

In [0]:
%sql
CREATE TABLE PiotrT_Demo.Minifigs_Identity
(
  Id bigint GENERATED ALWAYS AS IDENTITY,    -- automatically generates id for each new record
  LastModifiedDatetimeOriginal string,
  LastModifiedDate date,
  LastModifiedDatetime timestamp,
  Name string,
  MinifigType string,
  NumberOfParts int,
  ImageURL string,
  SetNumber string,
  SetURL string
)
-- LOCATION '' PATH FOR EXTERNAL LOCATION 

In [0]:
%sql
 INSERT INTO PiotrT_Demo.Minifigs_Identity(
     LastModifiedDatetimeOriginal,
     LastModifiedDate,
     LastModifiedDatetime,
     Name,
     MinifigType,
     NumberOfParts,
     ImageURL,
     SetNumber,
     SetURL
 )
 SELECT
  explodedArray.last_modified_dt AS LastModifiedDatetimeOriginal,
   to_date(explodedArray.last_modified_dt) AS LastModifiedDate,
   to_timestamp(explodedArray.last_modified_dt) AS LastModifiedDatetime,
   explodedArray.name AS Name,
   CASE
     WHEN explodedArray.name LIKE '%Toy%' THEN 'Toy'
     WHEN explodedArray.name LIKE '%Droid%' THEN 'Droid'
     ELSE 'Other'
   END AS MinifigType,
   cast(explodedArray.num_parts AS int) AS NumberOfParts,
   explodedArray.set_img_url AS ImageURL,
   explodedArray.set_num AS SetNumber,
   explodedArray.set_url AS SetURL
 FROM
   myData

In [0]:
%sql
SELECT * FROM PiotrT_Demo.Minifigs_Identity